# Supervised Learning - K Nearest Neighbors and Artificial Neural Network

## Introduction

We will be using customer churn data from the telecom industry. The data file is called 
`Orange_Telecom_Churn_Data.csv`. We will load this data together, do some preprocessing, and use K-nearest neighbors to predict customer churn based on account characteristics.

In [19]:
from __future__ import print_function
import os
data_path = ['DataSet']


* Begin by importing the data. Examine the columns and data.
* Notice that the data contains a state, area code, and phone number. Do you think these are good features to use when building a machine learning model? Why or why not? 

In [20]:
import pandas as pd

# Import the data using the file path
filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
data = pd.read_csv(filepath)

In [21]:
data.head(10)

,state,account_length,area_code,phone_number,intl_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churned
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
5,AL,118,510,391-8027,yes,no,0,223.4,98,37.98,...,101,18.75,203.9,118,9.18,6.3,6,1.70,0,False
6,MA,121,510,355-9993,no,yes,24,218.2,88,37.09,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,False
7,MO,147,415,329-9001,yes,no,0,157.0,79,26.69,...,94,8.76,211.8,96,9.53,7.1,6,1.92,0,False
8,LA,117,408,335-4719,no,no,0,184.5,97,31.37,...,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
9,WV,141,415,330-8173,yes,yes,37,258.6,84,43.96,...,111,18.87,326.4,97,14.69,11.2,5,3.02,0,False


In [22]:
data.shape

(5000, 21)

In [23]:
# Remove extraneous columns
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)

In [6]:
data.columns

Index(['account_length', 'intl_plan', 'voice_mail_plan',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls', 'churned'],
      dtype='object')

In [7]:
data.shape

(5000, 18)

* Notice that some of the columns are categorical data and some are floats. These features will need to be numerically encoded.
* Finally, K-nearest neighbors requires scaled data. Scale the data using one of the scaling methods

In [8]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])

In [9]:
# Mute the sklearn warning
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.preprocessing import MinMaxScaler

msc = MinMaxScaler()

data = pd.DataFrame(msc.fit_transform(data),  # this is an np.array, not a dataframe.
                    columns=data.columns)

* Separate the feature columns (everything except `churned`) from the label (`churned`). This will create two tables.
* Fit a K-nearest neighbors model with a value of `k=3` to this data and predict the outcome on the same data.

In [10]:
# Get a list of all the columns that don't contain the label
x_cols = [x for x in data.columns if x != 'churned']

# Split the data into two dataframes
X_data = data[x_cols]
y_data = data['churned']

# # alternatively:
# X_data = data.copy()
# y_data = X_data.pop('churned')

#### Train_Test_Split

In [11]:
# 70:30
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

In [12]:
# Now we can train a K-NN classifier
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn = knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

In [13]:
# Function to calculate the % of values that were correctly predicted

def accuracy(real, predict):
    return sum(real == predict) / float(real.shape[0])

In [14]:
print(accuracy(y_test, y_pred))

0.8993333333333333


In [15]:
# classification report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      1293
         1.0       0.75      0.40      0.52       207

   micro avg       0.90      0.90      0.90      1500
   macro avg       0.83      0.69      0.73      1500
weighted avg       0.89      0.90      0.89      1500



Let's see if we can improve the accuracy of our `K-NN` model

`BE CAREFUL OF OVERFITTING`

In [16]:
# Fitting KNN classifier to the training set

# This version of the model demos overfitting problem 

knn = KNeighborsClassifier(n_neighbors = 3, weights = 'distance')
knn.fit(X_data, y_data)
y_pred = knn.predict(X_data)

# Call function to calcuate the % of values that were correctly predicted
print(accuracy(y_data, y_pred))

1.0


In [17]:
# Fitting KNN classifier to the training set
# p = 1 Manhattan Distance (Default), p = 2 Euclidean Distance 
knn = KNeighborsClassifier(n_neighbors = 3, weights = 'uniform', metric = 'minkowski', p = 1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

# Call function to calcuate the % of values that were correctly predicted
print(accuracy(y_test, y_pred))

0.9126666666666666


### Artificial Neural Network
Lets try building a simple `ANN`

In [18]:
# Neural Network
# 1. Load Data.
# 2. Define Keras Model.
# 3. Compile Keras Model.
# 4. Fit Keras Model.
# 5. Evaluate Keras Model.
# 6. Make Predictions

from keras.models import Sequential 
from keras.layers import Dense

# load the dataset (already done )
# split into input (X) and output (y) variables (already done)

# define the keras model
model = Sequential()
model.add(Dense(17, input_dim=17, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=10)

# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
3500/3500 [==============================] - 1s 294us/step - loss: 0.4317 - acc: 0.8571
Epoch 2/150
3500/3500 [==============================] - 1s 192us/step - loss: 0.3887 - acc: 0.8571
Epoch 3/150
3500/3500 [==============================] - 1s 187us/step - loss: 0.3735 - acc: 0.8571
Epoch 4/150
3500/3500 [==============================] - 1s 208us/step - loss: 0.3655 - acc: 0.8577
Epoch 5/150
3500/3500 [==============================] - 1s 202us/step - loss: 0.3534 - acc: 0.8609
Epoch 6/150
3500/3500 [==============================] - 1s 198us/step - loss: 0.3480 - acc: 0.8583
Epoch 7/150
3500/3500 [==============================] - 1s 203us/step - loss: 0.3416 - acc: 0.8577
Epoch 8/150
3500/3500 [==============================] - 1s 201us/step - loss: 0.3344 - acc: 0.8606
Epoch 9/150
3500/3500 [==============================] - 1s 188us/step - loss: 0

3500/3500 [==============================] - 1s 183us/step - loss: 0.1850 - acc: 0.9354
Epoch 77/150
3500/3500 [==============================] - 1s 177us/step - loss: 0.1831 - acc: 0.9363
Epoch 78/150
3500/3500 [==============================] - 1s 171us/step - loss: 0.1843 - acc: 0.9354
Epoch 79/150
3500/3500 [==============================] - 1s 161us/step - loss: 0.1835 - acc: 0.9354
Epoch 80/150
3500/3500 [==============================] - 1s 174us/step - loss: 0.1836 - acc: 0.9349
Epoch 81/150
3500/3500 [==============================] - 1s 182us/step - loss: 0.1827 - acc: 0.9369
Epoch 82/150
3500/3500 [==============================] - 1s 225us/step - loss: 0.1844 - acc: 0.9351
Epoch 83/150
3500/3500 [==============================] - 1s 182us/step - loss: 0.1817 - acc: 0.9360
Epoch 84/150
3500/3500 [==============================] - 1s 182us/step - loss: 0.1836 - acc: 0.9323
Epoch 85/150
3500/3500 [==============================] - 1s 182us/step - loss: 0.1811 - acc: 0.9340
Epo